<a href="https://colab.research.google.com/github/WILLIANSCASACOLA/Listar_Arquivos_python/blob/main/UnirDataFrames.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [172]:
#Criar uma sessão spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("teste").getOrCreate()


In [173]:
#Subir dados para google:
from google.colab import files
uploaded = files.upload()

In [174]:
# Criar df_Consumer
# Esses arquivos estão aqui: D:\Aplicativos\Bancos\Bases
df_Consumer = spark.read.csv("Consumer.csv", header=True, inferSchema=True, sep=";",encoding="UTF8")
df_Corporate = spark.read.csv("Corporate.csv", header=True, inferSchema=True,sep=";",encoding="UTF8")
df_Home_Office = spark.read.csv("HomeOffice.csv", header=True, inferSchema=True,sep=";",encoding="UTF8")

In [175]:
#Para exibir tudo corretamente, use:
df_Consumer.show(truncate=False)

+--------------+-----------+----------+--------+-------------+-------+---------------+---------------+-------------------------------------------------------------------+-----------+----------+
|ID_Pedido     |Data_Pedido|ID_Cliente|Segmento|Pais         |Regiao |ID_Produto     |Categoria      |Nome_Produto                                                       |Valor_Venda|Quantidade|
+--------------+-----------+----------+--------+-------------+-------+---------------+---------------+-------------------------------------------------------------------+-----------+----------+
|CA-2016-152156|08/11/2016 |CG-12520  |Consumer|United States|South  |FUR-BO-10001798|Furniture      |Bush Somerset Collection Bookcase                                  |261.96     |NULL      |
|CA-2016-152156|08/11/2016 |CG-12520  |Consumer|United States|South  |FUR-CH-10000454|Furniture      |Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back        |731.94     |NULL      |
|US-2015-108966|11/10/2015 |SO

In [176]:
#Para exibir tudo corretamente, use:
df_Corporate.show(truncate=False)

+--------------+-----------+----------+---------+-------------+-------+---------------+---------------+------------------------------------------------------------------------+-----------+----------+
|ID_Pedido     |Data_Pedido|ID_Cliente|Segmento |Pais         |Regiao |ID_Produto     |Categoria      |Nome_Produto                                                            |Valor_Venda|Quantidade|
+--------------+-----------+----------+---------+-------------+-------+---------------+---------------+------------------------------------------------------------------------+-----------+----------+
|CA-2016-138688|12/06/2016 |DV-13045  |Corporate|United States|West   |OFF-LA-10000240|Office Supplies|Self-Adhesive Address Labels for Typewriters by Universal               |14.62      |2         |
|CA-2016-137330|09/12/2016 |KB-16585  |Corporate|United States|Central|OFF-AR-10000246|Office Supplies|Newell 318                                                              |19.46      |7         |


In [177]:
#Para exibir tudo corretamente, use:
df_Home_Office.show(truncate=False)

+--------------+-----------+----------+-----------+-------------+-------+---------------+---------------+----------------------------------------------------------------------------+-----------+----------+
|ID_Pedido     |Data_Pedido|ID_Cliente|Segmento   |Pais         |Regiao |ID_Produto     |Categoria      |Nome_Produto                                                                |Valor_Venda|Quantidade|
+--------------+-----------+----------+-----------+-------------+-------+---------------+---------------+----------------------------------------------------------------------------+-----------+----------+
|US-2015-118983|22/11/2015 |HP-14815  |Home Office|United States|Central|OFF-AP-10002311|Office Supplies|Holmes Replacement Filter for HEPA Air Cleaner, Very Large Room, HEPA Filter|68.81      |5         |
|US-2015-118983|22/11/2015 |HP-14815  |Home Office|United States|Central|OFF-BI-10000756|Office Supplies|Storex DuraTech Recycled Plastic Frosted Binders                       

In [178]:
#Juntar todos os df em apenas um só
df_all = df_Consumer.union(df_Corporate).union(df_Home_Office)

In [179]:
#Colar uma coluna chamada "Dt_ingest" no df_all
from pyspark.sql.functions import current_timestamp
df_all = df_all.withColumn("Dt_ingest", current_timestamp())

In [180]:
#Para exibir tudo corretamente, use:
df_all.show(truncate=False)

+--------------+-----------+----------+--------+-------------+-------+---------------+---------------+-------------------------------------------------------------------+-----------+----------+--------------------------+
|ID_Pedido     |Data_Pedido|ID_Cliente|Segmento|Pais         |Regiao |ID_Produto     |Categoria      |Nome_Produto                                                       |Valor_Venda|Quantidade|Dt_ingest                 |
+--------------+-----------+----------+--------+-------------+-------+---------------+---------------+-------------------------------------------------------------------+-----------+----------+--------------------------+
|CA-2016-152156|08/11/2016 |CG-12520  |Consumer|United States|South  |FUR-BO-10001798|Furniture      |Bush Somerset Collection Bookcase                                  |261.96     |NULL      |2025-06-08 22:15:44.532846|
|CA-2016-152156|08/11/2016 |CG-12520  |Consumer|United States|South  |FUR-CH-10000454|Furniture      |Hon Deluxe Fab

In [181]:
#Mostrar total geral de linha no "df_all"
df_all.count()

9994

In [182]:
df_all.show(truncate=False)

+--------------+-----------+----------+--------+-------------+-------+---------------+---------------+-------------------------------------------------------------------+-----------+----------+--------------------------+
|ID_Pedido     |Data_Pedido|ID_Cliente|Segmento|Pais         |Regiao |ID_Produto     |Categoria      |Nome_Produto                                                       |Valor_Venda|Quantidade|Dt_ingest                 |
+--------------+-----------+----------+--------+-------------+-------+---------------+---------------+-------------------------------------------------------------------+-----------+----------+--------------------------+
|CA-2016-152156|08/11/2016 |CG-12520  |Consumer|United States|South  |FUR-BO-10001798|Furniture      |Bush Somerset Collection Bookcase                                  |261.96     |NULL      |2025-06-08 22:15:45.255295|
|CA-2016-152156|08/11/2016 |CG-12520  |Consumer|United States|South  |FUR-CH-10000454|Furniture      |Hon Deluxe Fab

In [183]:
#Mostrar o schema dos dados:
df_all.printSchema()

root
 |-- ID_Pedido: string (nullable = true)
 |-- Data_Pedido: string (nullable = true)
 |-- ID_Cliente: string (nullable = true)
 |-- Segmento: string (nullable = true)
 |-- Pais: string (nullable = true)
 |-- Regiao: string (nullable = true)
 |-- ID_Produto: string (nullable = true)
 |-- Categoria: string (nullable = true)
 |-- Nome_Produto: string (nullable = true)
 |-- Valor_Venda: string (nullable = true)
 |-- Quantidade: integer (nullable = true)
 |-- Dt_ingest: timestamp (nullable = false)



In [184]:
# Converte coluna "Valor_Venda" para integer
from pyspark.sql.types import DoubleType
df_all = df_all.withColumn("Valor_Venda", df_all["Valor_Venda"].cast(DoubleType()))

In [185]:
df_all.printSchema()

root
 |-- ID_Pedido: string (nullable = true)
 |-- Data_Pedido: string (nullable = true)
 |-- ID_Cliente: string (nullable = true)
 |-- Segmento: string (nullable = true)
 |-- Pais: string (nullable = true)
 |-- Regiao: string (nullable = true)
 |-- ID_Produto: string (nullable = true)
 |-- Categoria: string (nullable = true)
 |-- Nome_Produto: string (nullable = true)
 |-- Valor_Venda: double (nullable = true)
 |-- Quantidade: integer (nullable = true)
 |-- Dt_ingest: timestamp (nullable = false)



In [186]:
# Criar uma coluna chamada "Valor_Total_Vendas" que é "Valor_Venda" *
# "Quantidade" %% Gerar um excel com nome "Dados_Unidos" com a data de hoje no
df_all = df_all.withColumn("Valor_Total_Vendas", df_all["Valor_Venda"] * df_all["Quantidade"])

In [187]:
#Gerar um excel com nome "Dados_Unidos" com a data de hoje no nome
import datetime
data_hoje = datetime.date.today()

In [188]:
# Criar uma coluna nova chamada "Fleg_cisco" quando encontrar o valor
# "Nome_Produto" %Cisco% preecher a coluna "Fleg_cisco" com "1"
from pyspark.sql.functions import when, col
df_all = df_all.withColumn("Flag_cisco", when(col("Nome_Produto").like("%Cisco%"), 1).otherwise(0))
df_all = df_all.withColumn("Flag_Bush", when(col("Nome_Produto").like("%Bush%"), 1).otherwise(0))

df_all.show()

+--------------+-----------+----------+--------+-------------+-------+---------------+---------------+--------------------+-----------+----------+--------------------+------------------+----------+---------+
|     ID_Pedido|Data_Pedido|ID_Cliente|Segmento|         Pais| Regiao|     ID_Produto|      Categoria|        Nome_Produto|Valor_Venda|Quantidade|           Dt_ingest|Valor_Total_Vendas|Flag_cisco|Flag_Bush|
+--------------+-----------+----------+--------+-------------+-------+---------------+---------------+--------------------+-----------+----------+--------------------+------------------+----------+---------+
|CA-2016-152156| 08/11/2016|  CG-12520|Consumer|United States|  South|FUR-BO-10001798|      Furniture|Bush Somerset Col...|     261.96|      NULL|2025-06-08 22:15:...|              NULL|         0|        0|
|CA-2016-152156| 08/11/2016|  CG-12520|Consumer|United States|  South|FUR-CH-10000454|      Furniture|Hon Deluxe Fabric...|     731.94|      NULL|2025-06-08 22:15:...| 

In [189]:
df_all.show(truncate=False)

+--------------+-----------+----------+--------+-------------+-------+---------------+---------------+-------------------------------------------------------------------+-----------+----------+--------------------------+------------------+----------+---------+
|ID_Pedido     |Data_Pedido|ID_Cliente|Segmento|Pais         |Regiao |ID_Produto     |Categoria      |Nome_Produto                                                       |Valor_Venda|Quantidade|Dt_ingest                 |Valor_Total_Vendas|Flag_cisco|Flag_Bush|
+--------------+-----------+----------+--------+-------------+-------+---------------+---------------+-------------------------------------------------------------------+-----------+----------+--------------------------+------------------+----------+---------+
|CA-2016-152156|08/11/2016 |CG-12520  |Consumer|United States|South  |FUR-BO-10001798|Furniture      |Bush Somerset Collection Bookcase                                  |261.96     |NULL      |2025-06-08 22:15:46.2484

In [190]:
data_hoje = data_hoje.strftime("%d-%m-%Y")

In [191]:
print(data_hoje)

08-06-2025


In [192]:
#Exportar "df_all" para o excel
df_all.toPandas().to_excel(f'Dados_Unidos_{data_hoje}.xlsx')
print(f"Dados exportados com êxito 'Dados_Unidos_{data_hoje}.xlsx'")

Dados exportados com êxito 'Dados_Unidos_08-06-2025.xlsx'
